In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src import layer, losses
from src.model import Model

In [ ]:
data_df = pd.read_csv('energy_efficiency_data.csv')

data_x_df = data_df.drop('Heating Load', axis=1)
data_x_df = pd.get_dummies(data_x_df,
                           columns=['Orientation', 'Glazing Area Distribution'],
                           dtype=float)

data_df = pd.concat(
    [data_x_df, data_df['Heating Load']],
    axis=1,
)

data_df


In [ ]:
dataset = np.array(data_df)
np.random.shuffle(dataset)

train_ratio = 0.75
train_size = int(len(dataset) * train_ratio)

train_feature = dataset[:train_size, :-1]
test_feature = dataset[train_size:, :-1]
train_target = dataset[:train_size, -1:]
test_target = dataset[train_size:, -1:]

In [ ]:
def split_batch(x, y, batch_size):
    for i in range(0, len(x), batch_size):
        if i + batch_size < len(x):
            yield x[i:i + batch_size], y[i:i + batch_size]
        yield x[i:], y[i:]

In [ ]:
INPUT_FEATURES = train_feature.shape[1]
OUTPUT_FEATURES = 1

model = Model([
    layer.Linear(INPUT_FEATURES, 16),
    layer.ReLU(),
    layer.Linear(16, OUTPUT_FEATURES),
])

mse = losses.MSE()

In [ ]:
EPOCHES = 1000
BATCH_SIZE = 64

train_losses = []
for epoch in range(EPOCHES):

    losses = []
    for x, y in split_batch(train_feature, train_target, BATCH_SIZE):
        y_pred = model(x)
        losses.append(mse(y_pred, y))
        grad = mse.backward()
        model.backward(grad)
        model.update(0.001)

    train_losses.append(np.mean(losses))

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch}: {np.mean(losses)}')

In [ ]:
plt.plot(train_losses)

In [ ]:
y_pred = model(test_feature)
print(mse(y_pred, test_target))